In [ ]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
m = tf.keras.metrics.Accuracy()
m.update_state([1, 2, 3, 4], [1, 2, 3, 4])
print('Final result: ', m.result().numpy()) 

Final result:  1.0


In [1]:
from google.colab import files
files.upload()

Saving final_none.csv to final_none.csv


{'final_none.csv': b'SOURCE,TWEET,TIME,CONTENT,DISASTER_NOT,TYPE\r\nCAL_FIRE,1.43E+18,2021/8/15 14:38,"#CoyoteFire off State Highway 29 and Spruce Grove Road, Hidden Valley Lake in Lake County is 127 acres and 95% contained. @CALFIRELNU ",1,Fire\r\nCAL_FIRE,1.42E+18,2021/8/1 14:41,"#TobinFire off of Highway 46 East and Union Road, East of Paso Robles in San Luis Obispo County is 75 acres and 100% contained @CALFIRE_SLO ",1,Fire\r\nNWS,1.40E+18,2021/6/4 13:14,Severe thunderstorms will be possible from parts of central Montana to extreme northwestern Minnesota this afternoon and evening. Damaging wind gusts are expected to be the primary threat.Visit  for more. ,1,Storm\r\nCAL_FIRE,1.43E+18,2021/8/28 16:20,"#MonumentFire off Del Loma, CA near Monument Peak in Trinity county is 161,309 acres and 23% contained. Unified Command: @CALFIRESHU and @ShastaTrinityNF ",1,Fire\r\nNWS,1.43E+18,2021/8/30 18:11,"Flash flooding will continue through the afternoon hours in association with Tropical Sto

In [ ]:
import pandas as pd 
import numpy as np 
import re
import string
# from tensorflow import keras
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
# from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Dense, Dropout

from sklearn.preprocessing import LabelEncoder

import bert 
import bert.tokenization as tokenization


In [ ]:
def data_preprocess(disaster_df):

  def remove_emoji(text):
    emoji = re.compile("["
                        u"\U0001F600-\U0001F64F"  
                        u"\U0001F300-\U0001F5FF"  
                        u"\U0001F680-\U0001F6FF"  
                        u"\U0001F1E0-\U0001F1FF"  
                        u"\U00002702-\U000027B0"
                        u"\U000024C2-\U0001F251"
                        "]+", flags = re.UNICODE)
    return emoji.sub(r'', text)
  
  def remove_punct(text):
    mapping = str.maketrans('', '', string.punctuation)
    return text.translate(mapping)
  
  def remove_links(text):
    text = ''.join([x for x in text if x in string.printable])
    text = re.sub(r'https?://\S+|www\.\S+', "", text)
    return text


  disaster_df['CONTENT'] = disaster_df['CONTENT'].apply(lambda x: remove_emoji(x))
  disaster_df['CONTENT'] = disaster_df['CONTENT'].apply(lambda x: remove_punct(x))
  disaster_df['CONTENT'] = disaster_df['CONTENT'].apply(lambda x: remove_links(x))
  disaster_df['CONTENT'] = disaster_df['CONTENT'].apply(lambda x: str(x).lower())

In [ ]:
def bert_encode(text_inputs, tokenizer, max_length = 128):
  tokens = []
  masks = []
  segments = []

  for text in text_inputs:
    text = tokenizer.tokenize(text)
    text = text[:max_length - 2]
    bert_input = ["[CLS]"] + text + ["[SEP]"]
    padding_length = max_length - len(bert_input)
    padding = [0] * padding_length

    token = tokenizer.convert_tokens_to_ids(bert_input)
    token += padding
    mask = [1] * len(bert_input) + padding
    segment = [0] * max_length

    tokens.append(token)
    masks.append(mask)
    segments.append(segment)

  return np.array(tokens), np.array(masks), np.array(segments)

In [ ]:
def bert_model(bert_layer, max_length = 128):
  word_ids = Input(shape = (max_length,), dtype = tf.int32)
  masks = Input(shape = (max_length,), dtype = tf.int32)
  segments = Input(shape = (max_length,), dtype = tf.int32)
  # only use the sequence_output,and the valid sequence lenght is only 1( since we need only 1 indicator for classification)
  _, bert_layer_output = bert_layer([word_ids, masks, segments])
  
  
  x = Dense(40, activation = 'relu')(bert_layer_output[:, 0, :])
  x = Dropout(0.2)(x)
  out_label1 = Dense(7, activation = 'softmax',name='type_out')(x)
  out_label2 = Dense(2, activation = 'softmax',name='flag_out')(x)
  

  model = Model(inputs = [word_ids, masks, segments], outputs = [out_label1,out_label2])
  model.compile(
    optimizer = Adam(lr = 5e-6), 
    loss = {
      'type_out':'categorical_crossentropy',
      'flag_out':'categorical_crossentropy',
    },
    loss_weights = {
      'type_out': 1.,
      'flag_out': 0.5 # maybe 0.5, since it s much easier to converge
    },
   metrics = ['accuracy'])
  return model

In [ ]:
def main():
  # DATA loading
  disaster_df = pd.read_csv("final_none.csv",encoding="unicode_escape")
  data_preprocess(disaster_df)


  # encode the two labels 
  le1 = LabelEncoder()
  disaster_df['Category_Encoded'] = le1.fit_transform(disaster_df['TYPE'])
  
  le2 = LabelEncoder()
  disaster_df['DISASTER_NOT']= le2.fit_transform(disaster_df['DISASTER_NOT'])
  
  # pretrained bert model from tensorflow hub
  module_url = "https://hub.tensorflow.google.cn/tensorflow/bert_en_uncased_L-12_H-768_A-12/1"
  bert_layer = hub.KerasLayer(module_url, trainable=True)
  # do toekenization
  vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
  #sess = tf.compat.v1.Session()
  #with sess.as_default():
   # vocab_file = bert_layer.resolved_object.vocab_file.asset_path.eval()
   # do_lower_case = bert_layer.resolved_object.do_lower_case.eval()
  do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
  
  tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

  
  # define input with tokenization
  train_input = bert_encode(disaster_df.CONTENT.values, tokenizer)

  # label 1 
  train_label1_value = disaster_df.Category_Encoded.values
  train_label1 = tf.keras.utils.to_categorical(train_label1_value, num_classes=7,dtype = 'int32')

  # label 2 
  train_label2_value = disaster_df.DISASTER_NOT.values
  train_label2 =tf.keras.utils.to_categorical(train_label2_value, num_classes=2,dtype = 'int32')

  train_labels = [train_label1,train_label2]

  # declare the model
  model = bert_model(bert_layer)
  model.summary()



  validation_split = 0.2
  epochs = 7 
  batch_size = 16

  train_history = model.fit(train_input, train_labels, validation_split = validation_split, epochs = epochs, batch_size = batch_size)

In [ ]:
main()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
keras_layer_3 (KerasLayer)      [(None, 768), (None, 109482241   input_4[0][0]                    
                                                                 input_5[0][0]              

IndexError: index 5 is out of bounds for axis 0 with size 2

In [ ]:
  # TEST PART
test_str = ["tornado threats from nicholas should remain low tonight across and inland of mid and upper texas coastal areas"]
  #test_str = ['hopkinsfire off hopkins st and north state st near calpella in mendocino county']
  test_str_input = bert_encode(test_str, tokenizer)
PRED_outputs = model.predict(test_str_input)
  
PRED_FLAG = np.argmax(PRED_outputs[1], axis = 1)
FLAG_str = '' if not PRED_FLAG else 'not'
print('This message is {} disaster '.format(FLAG_str))

encoded_classes = le2.classes_
PRED_TYPE = np.argmax(PRED_outputs[0], axis =0)
print('Predict Type - {}'.format(encoded_classes[PRED_TYPE]))


This message is not disaster 
Predict Type - [0 0 0 0 0 0 0]
